In [1]:
import numpy as np
import os
import shutil

from pommerman.envs.v0 import Pomme
from pommerman.agents import SimpleAgent, BaseAgent
from pommerman.configs import ffa_v0_env
from pommerman.constants import BOARD_SIZE, GameType
from tensorforce.agents import PPOAgent
from tensorforce.execution import Runner
from tensorforce.contrib.openai_gym import OpenAIGym

/gpfs/hpchome/anton95/.conda/envs/pommerman/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/gpfs/hpchome/anton95/.conda/envs/pommerman/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
num_episodes = 30000
batching_capacity = 1000
save_seconds = 300
main_dir = './ppo/'
log_path = main_dir + 'logs/'
model_path = main_dir + 'model'

In [3]:
if not os.path.isdir(main_dir):
    os.mkdir(main_dir)
if os.path.isdir(log_path):
    shutil.rmtree(log_path, ignore_errors=True)
os.mkdir(log_path)

In [4]:
# Instantiate the environment
config = ffa_v0_env()
env = Pomme(**config["env_kwargs"])
env.seed(0)

# Create a Proximal Policy Optimization agent
network = dict(type='pomm_network.PommNetwork')
states = {
    "board": dict(shape=(BOARD_SIZE, BOARD_SIZE, 3, ), type='float'),
    "state": dict(shape=(3,), type='float')
}
saver = {
    "directory": model_path,
    "seconds": save_seconds,
    "load": os.path.isdir(model_path)
}
agent = PPOAgent(
    states=states,
    actions=dict(type='int', num_actions=env.action_space.n),
    network=network,
    batching_capacity=batching_capacity,
    step_optimizer=dict(
        type='adam',
        learning_rate=1e-4
    ),
    saver=saver
)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 0 into ./ppo/model/model.ckpt.


In [5]:
class TensorforceAgent(BaseAgent):
    def act(self, obs, action_space):
        pass
# Add 3 random agents
agents = []
for agent_id in range(3):
    agents.append(SimpleAgent(config["agent"](agent_id, config["game_type"])))

# Add TensorforceAgent
agent_id += 1
agents.append(TensorforceAgent(config["agent"](agent_id, config["game_type"])))
env.set_agents(agents)
env.set_training_agent(agents[-1].agent_id)
env.set_init_game_state(None)

In [6]:
class WrappedEnv(OpenAIGym):
    def __init__(self, gym, visualize=False):
        self.gym = gym
        self.visualize = visualize

    def execute(self, actions):
        if self.visualize:
            self.gym.render()

        obs = self.gym.get_observations()
        all_actions = self.gym.act(obs)
        all_actions.insert(self.gym.training_agent, actions)
        state, reward, terminal, _ = self.gym.step(all_actions)
        agent_state = WrappedEnv.featurize(state[self.gym.training_agent])
        agent_reward = reward[self.gym.training_agent]
        # If nobody die, use some "smart" reward
        if agent_reward == 0:
            agent_reward = self.gym.train_reward
        return agent_state, terminal, agent_reward

    def reset(self):
        obs = self.gym.reset()
        agent_obs = WrappedEnv.featurize(obs[3])
        return agent_obs

    @staticmethod
    def featurize(obs):
        def get_matrix(dict, key):
            res = dict[key]
            return res.reshape(res.shape[0], res.shape[1], 1).astype(np.float32)

        board = get_matrix(obs, 'board')
        teammate_position = None
        teammate = obs["teammate"]
        if teammate is not None:
            teammate = teammate.value
            if teammate > 10 and teammate < 15:
                teammate_position = np.argwhere(board == teammate)[0]
        else:
            teammate = None
        # My self - 11
        # Team mate - 12
        # Enemy - 13

        # Everyone enemy
        board[(board > 10) & (board < 15)] = 13
        # I'm not enemy
        my_position = obs['position']
        board[my_position[0], my_position[1], 0] = 11
        # Set teammate
        if teammate_position is not None:
            board[teammate_position[0], teammate_position[1], teammate_position[2]] = 12

        bomb_blast_strength = get_matrix(obs, 'bomb_blast_strength')
        bomb_life = get_matrix(obs, 'bomb_life')
        conv_inp = np.concatenate([board, bomb_blast_strength, bomb_life], axis=2)
        state = np.array([obs["ammo"], obs["blast_strength"], obs["can_kick"]]).astype(np.float32)
        return dict(board=conv_inp, state=state)

In [7]:
def episode_finished(r):
    if r.episode % 10 == 0:
        print("Finished episode {ep} after {ts} timesteps".format(ep=r.episode + 1, ts = r.timestep + 1))
        print("Episode reward: {}".format(r.episode_rewards[-1]))
        print("Average of last 10 rewards: {}".format(np.mean(r.episode_rewards[10:])))
    return True

In [ ]:
# Instantiate and run the environment for 5 episodes.
wrapped_env = WrappedEnv(env, False)
runner = Runner(agent=agent, environment=wrapped_env)
runner.run(num_episodes=num_episodes, episode_finished=episode_finished, max_episode_timesteps=env._max_steps)
print("Stats: ", runner.episode_rewards, runner.episode_timesteps, runner.episode_times)

try:
    runner.close()
except AttributeError as e:
    pass

Finished episode 11 after 509 timesteps
Episode reward: -1.041
Average of last 10 rewards: nan


/gpfs/hpchome/anton95/.conda/envs/pommerman/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/gpfs/hpchome/anton95/.conda/envs/pommerman/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Finished episode 21 after 1215 timesteps
Episode reward: -1.033
Average of last 10 rewards: -1.1287
Finished episode 31 after 1755 timesteps
Episode reward: -1.08
Average of last 10 rewards: -1.0877500000000002
Finished episode 41 after 2342 timesteps
Episode reward: -1.051
Average of last 10 rewards: -1.0775000000000001
Finished episode 51 after 2770 timesteps
Episode reward: -1.112
Average of last 10 rewards: -1.0662749999999999
Finished episode 61 after 3589 timesteps
Episode reward: -1.065
Average of last 10 rewards: -1.0710799999999998
Finished episode 71 after 4270 timesteps
Episode reward: -1.095
Average of last 10 rewards: -1.0727499999999999
Finished episode 81 after 4695 timesteps
Episode reward: -1.072
Average of last 10 rewards: -1.0671714285714284
Finished episode 91 after 5136 timesteps
Episode reward: -1.115
Average of last 10 rewards: -1.0688625
Finished episode 101 after 5899 timesteps
Episode reward: -1.058
Average of last 10 rewards: -1.0764333333333331
Finished epis

Finished episode 721 after 45280 timesteps
Episode reward: -1.057
Average of last 10 rewards: -1.0741732394366197
Finished episode 731 after 45738 timesteps
Episode reward: -1.137
Average of last 10 rewards: -1.0744194444444444
Finished episode 741 after 46238 timesteps
Episode reward: -1.061
Average of last 10 rewards: -1.0742205479452056
Finished episode 751 after 47109 timesteps
Episode reward: -1.1980000000000002
Average of last 10 rewards: -1.074372972972973
Finished episode 761 after 48084 timesteps
Episode reward: -0.7010000000000001
Average of last 10 rewards: -1.0748146666666667
Finished episode 771 after 48440 timesteps
Episode reward: -1.077
Average of last 10 rewards: -1.074775
Finished episode 781 after 48745 timesteps
Episode reward: -1.111
Average of last 10 rewards: -1.0746376623376623
Finished episode 791 after 49140 timesteps
Episode reward: -0.655
Average of last 10 rewards: -1.073575641025641
Finished episode 801 after 49494 timesteps
Episode reward: -1.053
Average 

INFO:tensorflow:Saving checkpoints for 88617 into ./ppo/model/model.ckpt.
Finished episode 1411 after 89472 timesteps
Episode reward: -1.065
Average of last 10 rewards: -1.0741828571428573
Finished episode 1421 after 90376 timesteps
Episode reward: -1.342
Average of last 10 rewards: -1.0744836879432624
Finished episode 1431 after 90829 timesteps
Episode reward: -1.112
Average of last 10 rewards: -1.0745577464788734
Finished episode 1441 after 91526 timesteps
Episode reward: -1.07
Average of last 10 rewards: -1.074323776223776
Finished episode 1451 after 92118 timesteps
Episode reward: -1.047
Average of last 10 rewards: -1.0745701388888889
Finished episode 1461 after 92931 timesteps
Episode reward: -1.2240000000000002
Average of last 10 rewards: -1.075353103448276
Finished episode 1471 after 93553 timesteps
Episode reward: -0.7690000000000001
Average of last 10 rewards: -1.0753027397260273
Finished episode 1481 after 94210 timesteps
Episode reward: -1.11
Average of last 10 rewards: -1.0

...

Finished episode 14271 after 888044 timesteps
Episode reward: -1.061
Average of last 10 rewards: -1.0716612201963533
Finished episode 14281 after 888451 timesteps
Episode reward: -1.087
Average of last 10 rewards: -1.0716379117028731
Finished episode 14291 after 888967 timesteps
Episode reward: -1.053
Average of last 10 rewards: -1.0716519607843136
Finished episode 14301 after 890125 timesteps
Episode reward: -1.1780000000000002
Average of last 10 rewards: -1.0717324702589224
Finished episode 14311 after 890912 timesteps
Episode reward: -1.058
Average of last 10 rewards: -1.0717037762237762
Finished episode 14321 after 891303 timesteps
Episode reward: -0.644
Average of last 10 rewards: -1.0716715583508036
Finished episode 14331 after 891781 timesteps
Episode reward: -1.034
Average of last 10 rewards: -1.0716519553072625
Finished episode 14341 after 892238 timesteps
Episode reward: -1.049
Average of last 10 rewards: -1.0716642009769715
Finished episode 14351 after 893320 timesteps
Episo

Finished episode 14941 after 930104 timesteps
Episode reward: -1.055
Average of last 10 rewards: -1.0718738111185533
Finished episode 14951 after 930776 timesteps
Episode reward: -1.2140000000000002
Average of last 10 rewards: -1.071913922356091
Finished episode 14961 after 931261 timesteps
Episode reward: -1.122
Average of last 10 rewards: -1.0718985284280935
Finished episode 14971 after 931861 timesteps
Episode reward: -1.1640000000000001
Average of last 10 rewards: -1.071916577540107
Finished episode 14981 after 932425 timesteps
Episode reward: -1.045
Average of last 10 rewards: -1.0718832331329327
Finished episode 14991 after 933165 timesteps
Episode reward: -1.071
Average of last 10 rewards: -1.0719074098798398
Finished episode 15001 after 933571 timesteps
Episode reward: -1.101
Average of last 10 rewards: -1.0719032021347565
Finished episode 15011 after 934757 timesteps
Episode reward: -1.2520000000000002
Average of last 10 rewards: -1.0719222666666666
Finished episode 15021 afte

Finished episode 15611 after 975227 timesteps
Episode reward: -1.057
Average of last 10 rewards: -1.0722746153846154
Finished episode 15621 after 976245 timesteps
Episode reward: -1.085
Average of last 10 rewards: -1.072206982703395
INFO:tensorflow:Saving checkpoints for 976880 into ./ppo/model/model.ckpt.
Finished episode 15631 after 977033 timesteps
Episode reward: -1.1580000000000001
Average of last 10 rewards: -1.0722095390524968
Finished episode 15641 after 977858 timesteps
Episode reward: -1.3850000000000002
Average of last 10 rewards: -1.0722360204734485
Finished episode 15651 after 978498 timesteps
Episode reward: -1.084
Average of last 10 rewards: -1.072283567774936
Finished episode 15661 after 979093 timesteps
Episode reward: -0.6619999999999999
Average of last 10 rewards: -1.0722305431309904
Finished episode 15671 after 980129 timesteps
Episode reward: -1.049
Average of last 10 rewards: -1.0723049808429117
Finished episode 15681 after 980594 timesteps
Episode reward: -1.042


Finished episode 16281 after 1015155 timesteps
Episode reward: -1.081
Average of last 10 rewards: -1.0720819913952058
INFO:tensorflow:Saving checkpoints for 1015373 into ./ppo/model/model.ckpt.
Finished episode 16291 after 1015795 timesteps
Episode reward: -1.046
Average of last 10 rewards: -1.072105405405405
Finished episode 16301 after 1016551 timesteps
Episode reward: -1.093
Average of last 10 rewards: -1.072141927562922
Finished episode 16311 after 1017286 timesteps
Episode reward: -1.4060000000000004
Average of last 10 rewards: -1.072158773006135
Finished episode 16321 after 1017955 timesteps
Episode reward: -1.031
Average of last 10 rewards: -1.07218467198038
Finished episode 16331 after 1018523 timesteps
Episode reward: -1.074
Average of last 10 rewards: -1.0721792892156863
Finished episode 16341 after 1019205 timesteps
Episode reward: -1.245
Average of last 10 rewards: -1.0721590324556032
Finished episode 16351 after 1019867 timesteps
Episode reward: -1.089
Average of last 10 r

Finished episode 16941 after 1057420 timesteps
Episode reward: -1.077
Average of last 10 rewards: -1.07228812758417
Finished episode 16951 after 1058271 timesteps
Episode reward: -1.062
Average of last 10 rewards: -1.0723133412042503
Finished episode 16961 after 1059154 timesteps
Episode reward: -1.067
Average of last 10 rewards: -1.0722945132743362
Finished episode 16971 after 1059576 timesteps
Episode reward: -1.05
Average of last 10 rewards: -1.0722972287735848
Finished episode 16981 after 1060382 timesteps
Episode reward: -1.2650000000000001
Average of last 10 rewards: -1.0723166175604006
Finished episode 16991 after 1061048 timesteps
Episode reward: -1.2400000000000002
Average of last 10 rewards: -1.0723025323910482
Finished episode 17001 after 1061706 timesteps
Episode reward: -1.1660000000000001
Average of last 10 rewards: -1.0723335491465569
Finished episode 17011 after 1062615 timesteps
Episode reward: -0.6620000000000001
Average of last 10 rewards: -1.0723377647058825
Finishe